In [ ]:
import numpy as np
import numqi
import torch
from utils import *
from tqdm import tqdm

In [ ]:
# W state
n = 5
W_state = n_qubit_W(n)
model = numqi.matrix_space.DetectCanonicalPolyadicRankModel([2]*n, 2)
model.set_target(W_state)
kwargs = dict(theta0='uniform', tol=1e-14, num_repeat=3, print_every_round=1, early_stop_threshold=1e-14)
theta_optim = numqi.optimize.minimize(model, **kwargs)
print(theta_optim.fun)

In [ ]:
# Dicke state
n = 7
k = 2
dicke_state = numqi.dicke.get_dicke_basis(n, 2)[k].reshape([2]*n)
model = numqi.matrix_space.DetectCanonicalPolyadicRankModel([2]*n, 1)
model.set_target(dicke_state)
kwargs = dict(theta0='uniform', tol=1e-14, num_repeat=3, print_every_round=1, early_stop_threshold=1e-14)
theta_optim = numqi.optimize.minimize(model, **kwargs)
print(f'numerical result: {theta_optim.fun}')
print(f'analytical result: {dicke_gm(n, k)}')
# abs error
print(f'abs error: {np.abs(theta_optim.fun - dicke_gm(n, k))}')

In [ ]:
# Detecting border rank of dicke state
n = 5
k = 2
dicke_state = numqi.dicke.get_dicke_basis(n, 2)[k].reshape([2]*n)
kwargs = dict(theta0='uniform', tol=1e-14, num_repeat=3, print_every_round=0, early_stop_threshold=1e-14)
for i in range(1, k+3):
    model = numqi.matrix_space.DetectCanonicalPolyadicRankModel([2]*n, i)
    model.set_target(dicke_state)
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    print(f'numerical result for E_{i+1}: {theta_optim.fun}')


In [ ]:
import matplotlib.pyplot as plt
plt.subplots()
kwargs = dict(theta0='uniform', tol=1e-14, num_repeat=3, print_every_round=0, early_stop_threshold=1e-14)
n = 5
k = 2
r_list = np.arange(2, n//2 + 2)
ret = []
dicke_state = numqi.dicke.get_dicke_basis(n, 2)[k].reshape([2]*n)
for r in r_list:
    model = numqi.matrix_space.DetectCanonicalPolyadicRankModel([2]*n, r-1)
    model.set_target(dicke_state)
    theta_optim = numqi.optimize.minimize(model, **kwargs)
    ret.append(theta_optim.fun)
plt.plot(r_list, ret, '-')
plt.xlabel('r')
plt.ylabel(r'$E_r(|D_n^k \rangle)$')

In [ ]:
import numpy as np
import numqi
import pickle
from tqdm import tqdm

kwargs = dict(theta0='uniform', tol=1e-14, num_repeat=3, print_every_round=0, early_stop_threshold=1e-14)
ret = []
N = 7
dataset = []

for n in tqdm(np.arange(2, N)):
    for k in np.arange(1, n//2+1):
        r_list = np.arange(2, k+3)
        ret = []
        dicke_state = numqi.dicke.get_dicke_basis(n, 2)[k].reshape([2]*n)
        for r in r_list:
            model = numqi.matrix_space.DetectCanonicalPolyadicRankModel([2]*n, r-1)
            model.set_target(dicke_state)
            theta_optim = numqi.optimize.minimize(model, **kwargs)
            ret.append(theta_optim.fun)
        dataset.append({'n': n, 'k': k, 'r_list': r_list, 'ret1': ret})

# Save the data to a pickle file
with open('border_rank.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
import matplotlib.pyplot as plt
import pickle

colors = ['#006BA4', '#FF800E', '#595959']
alpha = np.linspace(0.4, 1, N-2)

# Load the data from the pickle file
with open('border_rank.pkl', 'rb') as f:
    dataset = pickle.load(f)

# Create the subplot
plt.subplots()

# Plot the data
for data in dataset:
    n = data['n']
    k = data['k']
    r_list = data['r_list']
    ret1 = data['ret1']
    plt.plot(r_list, ret1, '-o', markersize=4, color=colors[k-1], alpha = alpha[n-2], label=f'n={n}, k={k}', linewidth=1)

plt.ylim([-0.03, 0.72])
plt.xticks(np.arange(2, 6))
plt.xlabel('r')
plt.ylabel(r'$E_r(|D_n^k \rangle)$')
plt.legend()

# Save the figure
plt.savefig('border_rank.pdf')